In [ ]:
'''
The Dynamical Adjustment PLS Code will Output Two Primary Data Arrays of Interest

i) Y_all_stations = This is the raw observed data
ii) Y_dynadj_all_sites = This is the dynamically adjusted data which has natural variability removed

This notebook really only requires two data inputs to run:
        i) annual_flow_stats = a dataframe that includes the raw observed data that will be dynamically adjusted (in this case Annual Maximum Streamflow Time-Series at Unique Streamflow Gage Stations)
        ii) Gridded Sea Level Pressure (SLP) which is used to determine if/how SLP anomalies or natural variability influence Qx1d 
    
'''

# Load Python Libraries

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import os
import sys
import time

from scipy import stats
from scipy.stats import ttest_ind_from_stats
import statistics
import pymannkendall as mk

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cftime

import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import mapclassify as mc
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

from sklearn.cross_decomposition import PLSRegression   
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score  
from sklearn.model_selection import cross_val_predict  
from sklearn.decomposition import PCA
from scipy import signal

# reduce warnings (less pink)
import warnings
warnings.filterwarnings('ignore')

# Functions

In [9]:
def detrend_X(X, x_for_plots, num_years = 50, Detrend_or_Filter = 'Detrend'):

    if Detrend_or_Filter == 'None':
        X_for_corr_map = X

        pass
    
    elif Detrend_or_Filter == 'Linear_Detrend':
        model = LinearRegression()

        # Detrend X
        X_for_corr_map = np.zeros((X.shape[0],X.shape[1]))
        temp_X_axis = x_for_plots.reshape(len(x_for_plots),1)
        for grid_cell in range(0,X.shape[1]):
            temp_X = X[:,grid_cell]
            model.fit(temp_X_axis, temp_X)
            X_for_corr_map[:,grid_cell] = temp_X - model.predict(temp_X_axis)

    elif Detrend_or_Filter == 'Diff_Detrend':
        # detrending leads to (Years - 1) values
        # detrend by differencing: https://machinelearningmastery.com/time-series-trends-in-python/

        # Detrend X
        X_for_corr_map = np.zeros((X.shape[0]-1,X.shape[1]))
        for grid_cell in range(0,X.shape[1]):
            X_temp_grid_cell = []
            for yr in range(1, X.shape[0]):
                value = X[yr,grid_cell] - X[yr - 1,grid_cell]
                X_temp_grid_cell.append(value)
            X_for_corr_map[:,grid_cell] = X_temp_grid_cell

    elif Detrend_or_Filter == 'Filter':

        # set-up filter with 15-yr high pass
        sampleRate = 15 / num_years
        Nyquist_frequency = sampleRate/2
        sos = signal.butter(N=1, Wn=Nyquist_frequency, btype='highpass',output='sos')

        # Filter X
        X_for_corr_map = np.zeros((X.shape[0],X.shape[1]))
        for grid_cell in range(0,X.shape[1]):
            X_grid_cell_series = X[:,grid_cell]
            X_for_corr_map[:,grid_cell] = signal.sosfilt(sos, X_grid_cell_series)
            

    return X_for_corr_map


def detrend_Y(Y, x_for_plots, num_years = 50, Detrend_or_Filter = 'Detrend'):
    if Detrend_or_Filter == 'None':
        Y_for_corr_map = Y

    elif Detrend_or_Filter == 'Linear_Detrend':
        model = LinearRegression()
        
        # Detrend Y
        model.fit(x_for_plots.reshape(len(x_for_plots),1), Y)
        Y_for_corr_map = Y - model.predict(x_for_plots.reshape(len(x_for_plots),1))
        
    elif Detrend_or_Filter == 'Diff_Detrend':
        # detrending leads to (Years - 1) values
        # detrend by differencing: https://machinelearningmastery.com/time-series-trends-in-python/

        # Detrend Y
        Y_for_corr_map_list = []
        for yr in range(1, len(Y)):
            value = Y[yr] - Y[yr - 1]
            Y_for_corr_map_list.append(value)
        Y_for_corr_map = np.array(Y_for_corr_map_list)
        #Y_for_corr_map = Y_for_corr_map.reshape(Y_for_corr_map.shape[0])
        
    elif Detrend_or_Filter == 'Filter':

        # set-up filter with 15-yr high pass
        sampleRate = 15 / num_years
        Nyquist_frequency = sampleRate/2
        sos = signal.butter(N=1, Wn=Nyquist_frequency, btype='highpass',output='sos')

        # Filter Y
        Y_for_corr_map = signal.sosfilt(sos, Y)
        
    return Y_for_corr_map

## Takes 2/3rds the time as using stats.pearsonr: 

def np_pearson_cor(x, y):
    xv = x - x.mean(axis=0)
    yv = y - y.mean(axis=0)
    xvss = (xv * xv).sum(axis=0)
    yvss = (yv * yv).sum(axis=0)
    result = np.matmul(xv.transpose(), yv) / np.sqrt(np.outer(xvss, yvss))
    # bound the values to -1 to 1 in the event of precision issues
    return np.maximum(np.minimum(result, 1.0), -1.0)

#for reference steps needed if using stats.pearsonr       
# corr1 = stats.pearsonr(Y_for_corr_map,X_for_corr_map[:,i])
#Y_corr_1[i]=corr1[0]                 # returns r and p (want r)


In [10]:
def PLS_Pass(X,Y, tmp_x_for_plots, num_years = 50, Weight_Lat = False):
    
    
    Y_for_corr_map = detrend_Y(Y, tmp_x_for_plots, Detrend_or_Filter = Detrend_or_Filter)
    # Get Filtered X for Correlation Matrix
    X_for_corr_map = detrend_X(X, tmp_x_for_plots, Detrend_or_Filter = Detrend_or_Filter)

    #Calculate correlation matrices of SLP and SNOTEL
    XY_corr = np.ones((X.shape[1]))
    for i in range(0,X.shape[1]):
        XY_corr[i] = np_pearson_cor(X_for_corr_map[:,i],Y_for_corr_map)

    if Weight_Lat == True:
        # # Christian 2016/Smoliak 2015
        # Before Projecting X onto Correlation Matrix (W), Weight W (or X) by Cosine of Lat and Standardize 

        ## populate weight array
        XY_corr_weighted_all_vars = np.ones(X_Flattened_Length)
        for grid_cells in range(0,len(X_var_list)):
            XY_corr_weighted = np.ones((X_dim_1,X_dim_2))
            XY_corr_reshape = XY_corr[(X_dim_1*X_dim_2)*grid_cells:(X_dim_1*X_dim_2)*(grid_cells+1)].reshape(X_dim_1,X_dim_2)
            for long in range(0,XY_corr_reshape.shape[1]):
                XY_corr_weighted[:,long] = XY_corr_reshape[:,long] * weights

            XY_corr_weighted = XY_corr_weighted.reshape(X_dim_1*X_dim_2)

            # Standardize
            XY_corr_weighted /= XY_corr_weighted.std()
            
            # Add weighted correlation for variable 
            XY_corr_weighted_all_vars[(X_dim_1*X_dim_2)*grid_cells:(X_dim_1*X_dim_2)*(grid_cells+1)] = XY_corr_weighted
        
    else:
        XY_corr_weighted_all_vars = XY_corr
        
    XY_corr_weighted = XY_corr_weighted_all_vars
    
    return XY_corr_weighted

In [11]:
### Lots of Code for Simply Loading Sea Level Pressure (SLP) Data
### Code was previously set-up to also load/use geopotential height as a predictor,
### but has since been modified and is only up-to-date to use SLP

def Load(X_var_list, Domain_Extent, Center=True,Scale=False):
    
    '''
    Domain Extent Options: Siler, Lehner, or E_Pacific
    '''
    
    if Domain_Extent == 'Lehner':
        Domain_Extension = 'Lehner_Domain'  # 20-90N, 180-10 W (includes eastern pacific and atlantic)
    elif Domain_Extent == 'Siler':
        Domain_Extension = '110E_290E_0N_60N'  # supposedly includes western and eastern pacific
    elif Domain_Extent == 'E_Pacific':
        Domain_Extension = '0_to_60N_180_to_10w_Domain'  # mostly just includes the eastern pacific
        
        
    ### Get XY Dims
    if Deg_Quarter_or_One == 'Quarter':
        nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/Sept_May_mean_mslp_{Domain_Extension}.nc')
    elif Deg_Quarter_or_One == 'One':
        nov_march_mean_mslp = xr.open_dataset('/home/bbass/DATA/ERA5/Final/Sept_May_mean_mslp_110E_290E_0N_60N_1Degree.nc')
    X_dim_1 = len(nov_march_mean_mslp.latitude)
    X_dim_2 = len(nov_march_mean_mslp.longitude)
    
    ## Get Flattened Length to feed to other parts of code
    X_Flattened_Length = X_dim_1*X_dim_2*len(X_var_list)
    
    ### Create Array where X Data and Weights will be Stored
    All_X_Vars = np.zeros((num_years,X_dim_1*X_dim_2*len(X_var_list)))
    weights = np.zeros((X_dim_1*len(X_var_list)))
    
    ### Get Weights for adjusting area by latitude
    weights = np.cos(np.deg2rad(nov_march_mean_mslp.latitude))
    weights = weights.values # array with dimension of latitude
    
    # keeps track of how to insert variable into flattened All_X_Vars array
    var = 0
    
    if 'SLP' in X_var_list:

        if (use == 'Annual_1day_Max_fract_of_mean') or (use == 'Fraction Avg April 1st SWE') or (use == 'Winter_1day_Max_fract_of_mean'):
            if Deg_Quarter_or_One == 'Quarter':
                nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/Sept_May_mean_mslp_{Domain_Extension}.nc')  # NOV-MARCH MEAN:             # '/home/bbass/DATA/ERA5/Final/wy_mean_mslp_110E_290E_0N_75N.nc'
            elif Deg_Quarter_or_One == 'One':
                nov_march_mean_mslp = xr.open_dataset('/home/bbass/DATA/ERA5/Final/Sept_May_mean_mslp_110E_290E_0N_60N_1Degree.nc')
        elif (use == 'Annual_Mean_fract_of_mean') or (use == 'Min_30day_Mean_Flow_fract_of_mean'):
            nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/wy_mean_mslp_{Domain_Extension}.nc')
        elif use == 'Snowmelt_1day_Max_fract_of_mean':
            nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/Nov_July_mean_mslp_{Domain_Extension}.nc')
        elif use == 'Fall_Mean_fract_of_mean':
            nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/Sept_Nov_mean_mslp_{Domain_Extension}.nc')
        elif use == 'Winter_Mean_fract_of_mean':
            nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/Dec_Feb_mean_mslp_{Domain_Extension}.nc') 
        elif use == 'Spring_Mean_fract_of_mean':
            nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/Dec_May_mean_mslp_{Domain_Extension}.nc')
        elif use == 'Summer_Mean_fract_of_mean':
            nov_march_mean_mslp = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/Dec_Aug_mean_mslp_{Domain_Extension}.nc')
            
        nov_march_mean_mslp = nov_march_mean_mslp.sel(wy=slice(Start_Year,End_Year))  # start_end year defined when getting snotel data

        X_dim_1 = len(nov_march_mean_mslp.latitude)
        X_dim_2 = len(nov_march_mean_mslp.longitude)
        X = nov_march_mean_mslp.msl.values

        # PLS code centers already
        if Center == True: 
            X_mean = X.mean(axis=0)  ## gets mean for each grid cell across all years
            X -= X_mean

        ## Scale Data (code below is from source code of PLS)
        if Scale == True:
            X_std = X.std(axis=0, ddof=1)
            X_std[X_std == 0.0] = 1.0
            X /= X_std

        X = X.reshape(num_years,X_dim_1*X_dim_2)
        
        All_X_Vars[:,0:X_dim_1*X_dim_2] = X
        
        var+=1
    
    if ('z500' in X_var_list) or ('z250' in X_var_list): 

        if (use == 'Annual_1day_Max_fract_of_mean') or (use == 'Fraction Avg April 1st SWE'):
            nov_march_mean_geop_250_500 = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/nov_march_mean_geop_{Domain_Extension}_Minus_Global_Mean.nc') #NOV-MARCH MEAN: '/home/bbass/DATA/ERA5/Geopotential_Height/Processed/nov_march_mean_geop_110E_290E_Minus_Global_Mean.nc'
        elif (use == 'Annual_Mean_fract_of_mean') or (use == 'Min_30day_Mean_Flow_fract_of_mean'):
            nov_march_mean_geop_250_500 = xr.open_dataset(f'/home/bbass/DATA/ERA5/Final/wy_mean_geop_{Domain_Extension}_Minus_Global_Mean.nc')
        nov_march_mean_geop_250_500 = nov_march_mean_geop_250_500.sel(wy=slice(Start_Year,End_Year))  # start_end year defined when getting snotel data
        nov_march_mean_geop_250 = nov_march_mean_geop_250_500.sel(level=250)
        nov_march_mean_geop_500 = nov_march_mean_geop_250_500.sel(level=500)

        ## z250
        X_geop_250 = nov_march_mean_geop_250.z.values
        X_geop_250 = X_geop_250.reshape(num_years,X_dim_1*X_dim_2)  ## keep wy dimension, flatten lat/long dimension

        ## z500
        X_geop_500 = nov_march_mean_geop_500.z.values
        X_geop_500 = X_geop_500.reshape(num_years,X_dim_1*X_dim_2)  ## keep wy dimension, flatten lat/long dimension

        # PLS code centers already
        if Center == True: 
            # Center Data since have different features
            X_geop_250_mean = X_geop_250.mean(axis=0)  ## gets mean for each grid cell across all years
            X_geop_250 -= X_geop_250_mean

            # Center Data since have different features
            X_geop_500_mean = X_geop_500.mean(axis=0)  ## gets mean for each grid cell across all years
            X_geop_500 -= X_geop_500_mean

        ## Scale Data (code below is from source code of PLS)
        if Scale == True:
            X_std = X_geop_250.std(axis=0, ddof=1)
            X_std[X_std == 0.0] = 1.0
            X_geop_250 /= X_std

            X_std = X_geop_500.std(axis=0, ddof=1)
            X_std[X_std == 0.0] = 1.0
            X_geop_500 /= X_std
            
            
        if 'z500' in X_var_list:
            All_X_Vars[:,X_dim_1*X_dim_2:(X_dim_1*X_dim_2)*2] = X_geop_500
            var+=1
            
        if 'z250' in X_var_list:
            if var == 1:
                All_X_Vars[:,X_dim_1*X_dim_2:(X_dim_1*X_dim_2)*2] = X_geop_250
            elif var == 2:
                All_X_Vars[:,(X_dim_1*X_dim_2)*2:(X_dim_1*X_dim_2)*3] = X_geop_250

    X = All_X_Vars
        
        
    return X, X_Flattened_Length, X_dim_1, X_dim_2, weights

## Load Predictand (Y) Data (Annual Maximum Streamflow (Qx1d))

In [ ]:
annual_flow_stats = pd.read_csv('/home/bbass/DATA/Natural_Flow/Dyn_Adj_Analysis/Continuous_Data_1970_to_2020/annual_stats.csv',index_col='GageID')
Metadata = pd.read_csv('/home/bbass/DATA/Natural_Flow/Dyn_Adj_Analysis/Continuous_Data_1970_to_2020/METADATA.txt',index_col='GageID')

# PLS: Raw Code

In [3]:
######## USER-DEFINED #######
Single_Station = False
station_index_of_interest = 9034900   ## only defined if Single_Station == False
Center = True               ## Whether or not to center/scale X,Y Data
Scale =  False              ## Whether or not to center/scale X,Y Data
log_transform = False       ## Whether or not to perform log transform of Y Data
Num_Passes = 2              ## define how many passes want to make
Detrend_or_Filter = 'Linear_Detrend'   # 'Filter', 'Linear_Detrend', Diff_Detrend', 'FilterY_DetrendX', 'None'
X_var_list = ['SLP']        ## code was previously able to incorporate Geopotential Height, but currently set-up to work with SLP as predictor only
Domain_Extent = 'Siler'     ## Siler (W and E Pacific), Lehner (E Pacific and Atlantic), or E_Pacific
Deg_Quarter_or_One = 'One'  ## Can either define 'Quarter' or 'One' to get ERA5 SLP at x-degree...this just points to the path
ols_or_sen = 'sen'          ## Use Sen's Slope or OLS when determining Slope of Qx1d Trends

Start_Year = 1970   
End_Year = 2020
input_df = annual_flow_stats           ## dataframe that contains Y data want to use
use = 'Annual_1day_Max_fract_of_mean'  ## Column in Dataframe want to use as Y data (code set-up to use Qx1d)
######## USER-DEFINED #######

num_years = (End_Year - Start_Year)+1
x_for_plots = np.arange(Start_Year,End_Year+1)

### Get Y ###
# get station/basin list
_, idx = np.unique(input_df.index, return_index=True)
station_list = input_df.index[np.sort(idx)]

## Get Y Data into array format for PLS
Y_all_stations = np.zeros((num_years,len(station_list)))
i = 0
for station in station_list:
    # rows = years, columns = different stations
    Y_all_stations[:,i] = input_df[input_df.index == station][f'{use}'].values
    i+=1

if Single_Station == True:
    # Get data for Single Station
    Y_all_stations = input_df[input_df.index == station_index_of_interest][f'{use}'].values
    Y_all_stations = Y_all_stations.reshape(num_years,1)
### Get Y ###


# Get X
X, X_Flattened_Length, X_dim_1, X_dim_2, weights = Load(X_var_list, Domain_Extent, Center=True,Scale=False)

#### INITIALIZE ARRAYS WHERE RESULTS ARE STORED ###
Y_dynadj_Pass_1_all_sites = np.zeros((Y_all_stations.shape[0],Y_all_stations.shape[1]))
if Num_Passes > 1:
    Y_dynadj_Pass_2_all_sites = np.zeros((Y_all_stations.shape[0],Y_all_stations.shape[1]))
if Num_Passes == 3:
    Y_dynadj_Pass_3_all_sites = np.zeros((Y_all_stations.shape[0],Y_all_stations.shape[1]))
Y_dynadj_all_sites = np.zeros((Y_all_stations.shape[0],Y_all_stations.shape[1]))
Y_dynadj_all_sites = np.zeros((Y_all_stations.shape[0],Y_all_stations.shape[1]))
SLP_contribution_all_sites = np.zeros((Y_all_stations.shape[0],Y_all_stations.shape[1]))

perc_var_expalained_all_sites_1st_pass = np.zeros(Y_all_stations.shape[1])
if Num_Passes > 1:
    perc_var_expalained_all_sites_2nd_pass = np.zeros(Y_all_stations.shape[1])
if Num_Passes == 3:
    perc_var_expalained_all_sites_3rd_pass = np.zeros(Y_all_stations.shape[1])
# for overall variance
perc_var_expalained_all_sites = np.zeros(Y_all_stations.shape[1])

Y_corr_map_1 = np.zeros((X_Flattened_Length,Y_all_stations.shape[1]))
if Num_Passes > 1:
    Y_corr_map_2 = np.zeros((X_Flattened_Length,Y_all_stations.shape[1]))
if Num_Passes == 3:
    Y_corr_map_3 = np.zeros((X_Flattened_Length,Y_all_stations.shape[1]))

#### INITIALIZE ARRAYS WHERE RESULTS ARE STORED ###
# LOOP THROUGH EACH STATION AND APPLY PLS

current_iteration = 0
print_statement = round(Y_all_stations.shape[1] / 10)
perc_10 = 0

start_time = time.time()
for station in range(0,Y_all_stations.shape[1]):
    
    print(f'{station+1} / {Y_all_stations.shape[1]}')
    
    Y = Y_all_stations[:,station]
    
    ########### PLS #####################

    current_pass = 1
    ####### FIRST PASS #########
    #print(f'Pass {current_pass}')

    #XY_corr_1, SLP_contribution_1, Y_adj_1, t_1 = PLS_Pass(X,Y)   
    
    # Using Leave One Out CV
    Y_adj_1 = np.zeros(num_years)
    SLP_contribution_1 = np.zeros(num_years)
    for iyear in range(num_years):
#         if iyear%5 == 0:
#             print(f'{iyear} / {num_years}')
        # Leave One Out CV: fit is made using all other years of data
        X_tmp = np.delete(X, iyear, 0)  ### 0 = axis
        Y_tmp = np.delete(Y, iyear)
        tmp_x_for_plots = np.delete(x_for_plots,iyear)
        # Leave One Out CV: prediction is made using single year of data
        # Y_predict is the true value
        X_left_out = X[iyear,:].reshape(1,X.shape[1])
        Y_left_out = Y[iyear]
        XY_corr_weighted = PLS_Pass(X_tmp, Y_tmp, tmp_x_for_plots) 
        
        ### Make Prediction for Single Year using Correlation Matrix obtained from all other years
        t_scores = np.ones(num_years)
        for l in range(0,num_years):
            t_scores[l]=np.mean(XY_corr_weighted*X[l,:])

        #Calculate linear regression of t versus data being predicted
        reg1=stats.linregress(t_scores,Y)  

        #Calculate SLP estimates of SNOTEL data
        SLP_contribution_single_year = np.multiply(reg1[0],t_scores)

        #Subtract SLP estimates from SNOTEL data to yield dynamically-adjusted SNOTEL data
        Y_adj_single_year = np.subtract(Y,SLP_contribution_single_year)

        # get year that was predicted
        SLP_contribution_single_year = SLP_contribution_single_year[iyear]
        Y_adj_single_year = Y_adj_single_year[iyear]
        
        SLP_contribution_1[iyear] = SLP_contribution_single_year
        Y_adj_1[iyear] = Y_adj_single_year
        #square_error_temp[iyear] = np.square((Y_predict - Y_left_out))
        

    # variables for plot
    Y_dynadj = Y_adj_1
    SLP_contribution = SLP_contribution_1

    ## NEEDED FOR SECOND PASS

    current_pass += 1
    if current_pass <= Num_Passes:   
        
        t_1 = np.ones(num_years)
        for l in range(0,num_years):
            t_1[l]=np.mean(XY_corr_weighted*X[l,:])

        # P = vector of regression coefficients for predictor variables X
        P_regression = np.ones((X_Flattened_Length))
        for i in range(0,X_Flattened_Length):
            reg_X = stats.linregress(t_1,X[:,i])
            P_regression[i] = reg_X[0]

        # subtract X from SLP time-series multiplied by regression coefficients of X
        tP_1 = np.dot(t_1.reshape(num_years,1),P_regression.reshape(X_Flattened_Length,1).T)
        X_adj = X - tP_1
        
#         # written out way to obtain tP rather than using matrix math
#         tP_long_form = np.ones((num_years,X.shape[1]))
#         for year in range(num_years):
#             for grid_cell in range(0,X.shape[1]):
#                 tP_long_form[year,grid_cell] = t_1[year] * P_regression[grid_cell]

        # ####### SECOND PASS #########
        #print(f'Pass {current_pass}')

        #XY_corr_2, SLP_contribution_2, Y_adj_2, t_2 = PLS_Pass(X_adj,Y_adj_1)
        
        # Using Leave One Out CV
        Y_adj_2 = np.zeros(num_years)
        SLP_contribution_2 = np.zeros(num_years)
        for iyear in range(num_years):
#             if iyear%5 == 0:
#                 print(f'{iyear} / {num_years}')
            # Leave One Out CV: fit is made using all other years of data
            X_tmp = np.delete(X_adj, iyear, 0)  ### 0 = axis
            Y_tmp = np.delete(Y_adj_1, iyear)
            tmp_x_for_plots = np.delete(x_for_plots,iyear)
            # Leave One Out CV: prediction is made using single year of data
            # Y_predict is the true value
            X_left_out = X_adj[iyear,:].reshape(1,X_adj.shape[1])
            Y_left_out = Y_adj_1[iyear]
            XY_corr_weighted = PLS_Pass(X_tmp, Y_tmp, tmp_x_for_plots) 

            ### Make Prediction for Single Year using Correlation Matrix obtained from all other years
            t_scores = np.ones(num_years)
            for l in range(0,num_years):
                t_scores[l]=np.mean(XY_corr_weighted*X_adj[l,:])

            #Calculate linear regression of t versus data being predicted
            reg1=stats.linregress(t_scores,Y_adj_1)  

            #Calculate SLP estimates of SNOTEL data
            SLP_contribution_single_year = np.multiply(reg1[0],t_scores)

            #Subtract SLP estimates from SNOTEL data to yield dynamically-adjusted SNOTEL data
            Y_adj_single_year = np.subtract(Y_adj_1,SLP_contribution_single_year)

            # get year that was predicted
            SLP_contribution_single_year = SLP_contribution_single_year[iyear]
            Y_adj_single_year = Y_adj_single_year[iyear]

            SLP_contribution_2[iyear] = SLP_contribution_single_year
            Y_adj_2[iyear] = Y_adj_single_year        
        
        
        

        # variables for plot
        Y_dynadj = Y_adj_2
        SLP_contribution = SLP_contribution_1 + SLP_contribution_2

    current_pass += 1
    if current_pass <= Num_Passes:

        # NEEDED FOR THIRD PASS
        
        t_2 = np.ones(num_years)
        for l in range(0,num_years):
            t_2[l]=np.mean(XY_corr_weighted*X[l,:])

        # P = vector of regression coefficients for predictor variables X
        P_regression = np.ones((X_Flattened_Length))
        for i in range(0,X_Flattened_Length):
            reg_X = stats.linregress(t_2,X_adj[:,i])
            P_regression[i] = reg_X[0]

        # subtract X from SLP time-series multiplied by regression coefficients of X
        tP_2 = np.dot(t_2.reshape(num_years,1),P_regression.reshape(X_Flattened_Length,1).T)
        X_adj_2 = X_adj - tP_2

        # ####### THIRD PASS #########
        #print(f'Pass {current_pass}')

        XY_corr_3, SLP_contribution_3, Y_adj_3, t_3 = PLS_Pass(X_adj_2,Y_adj_2)

        # variables for plot
        Y_dynadj = Y_adj_3
        SLP_contribution = SLP_contribution_1 + SLP_contribution_2 + SLP_contribution_3
    
    # Save Y_adjusted and SLP Contribution
    Y_dynadj_Pass_1_all_sites[:,station] = Y_adj_1
    Y_dynadj_Pass_1_all_sites[Y_dynadj_Pass_1_all_sites<0] = 0
    if Num_Passes > 1:
        Y_dynadj_Pass_2_all_sites[:,station] = Y_adj_2
        Y_dynadj_Pass_2_all_sites[Y_dynadj_Pass_2_all_sites<0] = 0
    if Num_Passes == 3:
        Y_dynadj_Pass_3_all_sites[:,station] = Y_adj_3
        Y_dynadj_Pass_3_all_sites[Y_dynadj_Pass_3_all_sites<0] = 0
        
    Y_dynadj_all_sites[:,station] = Y_dynadj
    SLP_contribution_all_sites[:,station] = SLP_contribution
    
    # Save Percent Variance Explained
    perc_var_expalained_all_sites_1st_pass[station] = np.var(SLP_contribution_1) / np.var(Y)
    if Num_Passes > 1:
        perc_var_expalained_all_sites_2nd_pass[station] = np.var(SLP_contribution_2) / np.var(Y)
    if Num_Passes == 3:
        perc_var_expalained_all_sites_3rd_pass[station] = np.var(SLP_contribution_3) / np.var(Y)
    # overall variance explained from all passes
    perc_var_expalained_all_sites[station] = np.var(SLP_contribution) / np.var(Y)
        
    current_iteration += 1

# If have any negative fraction of mean values replace with zero
Y_dynadj_all_sites[Y_dynadj_all_sites<0] = 0

# Get mean across stations
Y = np.mean(Y_all_stations,axis=1)
Y_dynadj = np.mean(Y_dynadj_all_sites,axis=1)
SLP_contribution = np.mean(SLP_contribution_all_sites,axis=1)

end_time = time.time()

print('Done')

In [2]:
### Plot a Single Stations Observed Qx1d Time-Series and Dynamically Adjusted Time-Series

fig,ax = plt.subplots(1,1)

station_index = 91
x = np.arange(1970,2021)
y_single_site = Y_dynadj_all_sites[:,station_index]

if ols_or_sen == 'sen':
    res = mk.hamed_rao_modification_test(y_single_site)
else:
    res = stats.linregress(x_reg, y_single_site)

last_num = res.intercept + res.slope*x_reg[-1]
first_num = res.intercept + res.slope*x_reg[0]
perc_change_dyn_adj = round(((last_num - first_num) / first_num)*100,1)
ax.plot(x,res.intercept + res.slope*x_reg,color='red',linestyle='--')
ax.plot(x,y_single_site,label=f'Adjusted: {perc_change_dyn_adj}%',color='red')

y_single_site = Y_all_stations[:,station_index]
if ols_or_sen == 'sen':
    res = mk.hamed_rao_modification_test(y_single_site)
else:
    res = stats.linregress(x_reg, y_single_site)

last_num = res.intercept + res.slope*x_reg[-1]
first_num = res.intercept + res.slope*x_reg[0]
perc_change_dyn_adj = round(((last_num - first_num) / first_num)*100,1)
ax.plot(x,res.intercept + res.slope*x_reg,color='black',linestyle='--')
ax.plot(x,y_single_site,label=f'Observed: {perc_change_dyn_adj}%',color='black')

ax.set_ylabel('Qx1d Anomaly')
ax.set_xlabel('Water Year')

ax.legend()